<a href="https://colab.research.google.com/github/Amirgh8080/Anlyzer/blob/main/Road_Accident_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Nessassary Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Data Loading

In [3]:
# prompt: use kaggle api /content/kaggle.json to download the dataset :datasets/sobhanmoosavi/us-accidents

!pip install kaggle

!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sobhanmoosavi/us-accidents
!unzip us-accidents.zip


cp: cannot stat '/content/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents
License(s): CC-BY-NC-SA-4.0
100% 650M/653M [00:04<00:00, 174MB/s]
100% 653M/653M [00:04<00:00, 147MB/s]
Archive:  us-accidents.zip
  inflating: US_Accidents_March23.csv  


In [ ]:
# Load dataset
df = pd.read_csv('/content/US_Accidents_March23.csv')

df.head()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [12]:
df.describe

<bound method NDFrame.describe of                 ID   Source  Severity           Start_Time  \
0              A-1  Source2         3  2016-02-08 05:46:00   
1              A-2  Source2         2  2016-02-08 06:07:59   
2              A-3  Source2         2  2016-02-08 06:49:27   
3              A-4  Source2         3  2016-02-08 07:23:34   
4              A-5  Source2         2  2016-02-08 07:39:07   
...            ...      ...       ...                  ...   
7728389  A-7777757  Source1         2  2019-08-23 18:03:25   
7728390  A-7777758  Source1         2  2019-08-23 19:11:30   
7728391  A-7777759  Source1         2  2019-08-23 19:00:21   
7728392  A-7777760  Source1         2  2019-08-23 19:00:21   
7728393  A-7777761  Source1         2  2019-08-23 18:52:06   

                    End_Time  Start_Lat   Start_Lng   End_Lat    End_Lng  \
0        2016-02-08 11:00:00  39.865147  -84.058723       NaN        NaN   
1        2016-02-08 06:37:59  39.928059  -82.831184       NaN        NaN   
2        2016-02-08 07:19:27  39.063148  -84.032608       NaN        NaN   
3        2016-02-08 07:53:34  39.747753  -84.205582       NaN        NaN   
4        2016-02-08 08:09:07  39.627781  -84.188354       NaN        NaN   
...                      ...        ...         ...       ...        ...   
7728389  2019-08-23 18:32:01  34.002480 -117.379360  33.99888 -117.37094   
7728390  2019-08-23 19:38:23  32.766960 -117.148060  32.76555 -117.15363   
7728391  2019-08-23 19:28:49  33.775450 -117.847790  33.77740 -117.85727   
7728392  2019-08-23 19:29:42  33.992460 -118.403020  33.98311 -118.39565   
7728393  2019-08-23 19:21:31  34.133930 -117.230920  34.13736 -117.23934   

         Distance(mi)  ... Roundabout Station   Stop Traffic_Calming  \
0               0.010  ...      False   False  False           False   
1               0.010  ...      False   False  False           False   
2               0.010  ...      False   False  False           False   
3               0.010  ...      False   False  False           False   
4               0.010  ...      False   False  False           False   
...               ...  ...        ...     ...    ...             ...   
7728389         0.543  ...      False   False  False           False   
7728390         0.338  ...      False   False  False           False   
7728391         0.561  ...      False   False  False           False   
7728392         0.772  ...      False   False  False           False   
7728393         0.537  ...      False   False  False           False   

        Traffic_Signal Turning_Loop Sunrise_Sunset Civil_Twilight  \
0                False        False          Night          Night   
1                False        False          Night          Night   
2                 True        False          Night          Night   
3                False        False          Night            Day   
4                 True        False            Day            Day   
...                ...          ...            ...            ...   
7728389          False        False            Day            Day   
7728390          False        False            Day            Day   
7728391          False        False            Day            Day   
7728392          False        False            Day            Day   
7728393          False        False            Day            Day   

        Nautical_Twilight Astronomical_Twilight  
0                   Night                 Night  
1                   Night                   Day  
2                     Day                   Day  
3                     Day                   Day  
4                     Day                   Day  
...                   ...                   ...  
7728389               Day                   Day  
7728390               Day                   Day  
7728391               Day                   Day  
7728392               Day                   Day  
7728393               Day                   Day  

[7728394 rows

In [14]:
df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='mixed')

# Aggregate accidents by date (daily counts)
df['Date'] = df['Start_Time'].dt.date
daily_accidents = df.groupby('Date').size().reset_index(name='Accident_Count')


In [1]:
# Optional: Plot the time series
plt.figure(figsize=(12, 4))
plt.plot(daily_accidents['Date'], daily_accidents['Accident_Count'], label='Daily Accident Count')
plt.xlabel('Date')
plt.ylabel('Accident Count')
plt.title('Daily Accident Count Over Time')
plt.legend()
plt.show()


NameError: name 'plt' is not defined

In [ ]:
def create_lag_features(df, lag=7):
    """
    Create lag features for a time series DataFrame.
    Given a DataFrame with 'Accident_Count', this function creates lag_1, lag_2, ..., lag_{lag}
    as features to forecast the current day's accident count.
    """
    df_new = df.copy()
    for i in range(1, lag+1):
        df_new[f'lag_{i}'] = df_new['Accident_Count'].shift(i)
    df_new = df_new.dropna().reset_index(drop=True)
    return df_new

In [15]:
# Ensure the data is sorted by date
daily_accidents['Date'] = pd.to_datetime(daily_accidents['Date'])
daily_accidents = daily_accidents.sort_values('Date').reset_index(drop=True)

## Random Forest (RF)

In [ ]:
rf_model = RandomForestRegressor(
    n_estimators=100,  # Default hyperparameters (tune if needed)
    random_state=42
)
rf_model.fit(X_train, y_train)

## Support Vector Machine (SVM)

In [ ]:
svm_model = SVR(
    kernel='rbf',  # RBF kernel for non-linear regression
    C=1.0,         # Default hyperparameters (tune if needed)
    epsilon=0.1
)
svm_model.fit(X_train_scaled, y_train)

## Evaluate Models

In [ ]:
def evaluate_model(model, X_train, X_test, y_train, y_test, is_svm=False):
    # Predictions
    if is_svm:
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    else:
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)

    # Metrics
    metrics = {
        'MAE_train': mean_absolute_error(y_train, y_pred_train),
        'MSE_train': mean_squared_error(y_train, y_pred_train),
        'RMSE_train': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'R2_train': r2_score(y_train, y_pred_train),
        'MAE_test': mean_absolute_error(y_test, y_pred_test),
        'MSE_test': mean_squared_error(y_test, y_pred_test),
        'RMSE_test': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'R2_test': r2_score(y_test, y_pred_test)
    }
    return metrics

# Evaluate RF
rf_metrics = evaluate_model(rf_model, X_train, X_test, y_train, y_test)

# Evaluate SVM
svm_metrics = evaluate_model(svm_model, X_train_scaled, X_test_scaled, y_train, y_test, is_svm=True)

# Print results
print("Random Forest Metrics:")
print(pd.DataFrame([rf_metrics]))
print("\nSupport Vector Machine Metrics:")
print(pd.DataFrame([svm_metrics]))

## Results Interpretation